In [1]:
# Module 17 Deliverable 2

# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=94ce08ca86f1df8125d12cf78252dc1a0154200c77381e7649a6b2d739772d1e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-19 02:27:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-12-19 02:27:59 (11.1 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load Amazon Data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [5]:
# Drop null values
dropna_df = df.dropna()
dropna_df.show(10)
     

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [6]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   20009611|R100E0YCS06YNR|B00EFFW0HC|     277685961|       Battlefield 4|     Video Games|          5|            0|          0|   N|                Y|          Five Stars|     great thank you|2015-07-04 00:00:00|
|         US|   21036809|R100HR44BSB9I4|B007FTE2VW|     265303108|SimCity - Limited...| 

In [7]:
# Create the Vine DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100E0YCS06YNR|          5|            0|          0|   N|                Y|
|R100HR44BSB9I4|          1|            1|          1|   N|                Y|
|R100UABZOUF1JP|          1|            0|          0|   N|                Y|
|R101MOW8A5WC4J|          5|            0|          0|   N|                Y|
|R101X7KSBGIWF1|          1|            0|          0|   N|                Y|
|R101ZTSTDC8KQ4|          5|            0|          0|   N|                Y|
|R103Q4U55JZAD1|          5|            0|          0|   N|                Y|
|R103SW6BLR1HH6|          5|            0|          0|   N|                Y|
|R1048AZPZ1XQVY|          4|            0|          0|   N|                Y|
|R104C56FZRE21Q|          5|            0|          0|   N|     

In [9]:
# Filter the Vine DataFrame by those that had 20+ total votes
vine_df2 = vine_df.filter(vine_df["total_votes"] >= 20)
vine_df2.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
| RGOVXI764T1KO|          1|            5|         23|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
|R1L6ICM6QT2MIZ|          5|           16|         24|   N|                N|
|R1FYGNGA4R87BV|          1|            2|         45|   N|                N|
|R2IB86CLUPL35Y|          1|           16|         86|   N|                N|
|R1JAN381JW53ZT|          4|           26|         27|   N|     

In [10]:

# Filter the Vine DataFrame by those that had 50%+ of helpful votes
vine_df3 = vine_df2.filter(vine_df2["helpful_votes"]/vine_df2["total_votes"] >= 0.5)
vine_df3.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
|R1L6ICM6QT2MIZ|          5|           16|         24|   N|                N|
|R1JAN381JW53ZT|          4|           26|         27|   N|                Y|
|R33OTY8CWC7O6T|          2|           12|         24|   N|                N|
|R1S41I7PHZEMI1|          4|           75|        108|   N|                N|
|R3MFHNRMGPFIZ8|          5|           74|         93|   N|     

In [13]:
# Filter the Vine DataFrame by those that have a vine review
vine_yes_review_df = vine_df3.filter(vine_df3["vine"] == "Y")
vine_yes_review_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PU7JY8XL5M1N|          5|           19|         24|   Y|                N|
|R23WY68NT94WKE|          5|          125|        143|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
|R1UBDJF7PWB8BL|          2|           43|         59|   Y|                N|
|R1TNOMT3O5DZ0Z|          4|           27|         32|   Y|                N|
| RYBLWBAL5M1AY|          4|           68|         76|   Y|                N|
|R1IN4WPVLZB86G|          5|           34|         37|   Y|                N|
|R1QTD3QXCKGOG0|          5|           34|         34|   Y|                N|
|R2XT37DDSZ8ROK|          3|           28|         53|   Y|                N|
| REQZP78LTC0EY|          4|           21|         23|   Y|     

In [14]:
# Filter the Vine DataFrame by those that have a vine review
vine_no_review_df = vine_df3.filter(vine_df3["vine"] == "N")
vine_no_review_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
|R1L6ICM6QT2MIZ|          5|           16|         24|   N|                N|
|R1JAN381JW53ZT|          4|           26|         27|   N|                Y|
|R33OTY8CWC7O6T|          2|           12|         24|   N|                N|
|R1S41I7PHZEMI1|          4|           75|        108|   N|                N|
|R3MFHNRMGPFIZ8|          5|           74|         93|   N|     

In [15]:
# to import statistics 
from pyspark.sql.functions import col, avg

In [17]:
# Get summary statistics of those that had vine reviews
vine_yes_review_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|            94|                94|               94|                94|  94|               94|
|   mean|          null| 4.202127659574468|54.59574468085106|61.787234042553195|null|             null|
| stddev|          null|0.9791348741656417|65.26098459822538| 68.90976994895394|null|             null|
|    min|R10FO5UKKVZBK2|                 1|               14|                20|   Y|                N|
|    max| RYBLWBAL5M1AY|                 5|              347|               362|   Y|                N|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+



In [18]:
# Get summary statistics of those that did not have vine reviews
vine_no_review_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|  count|         40471|             40471|             40471|             40471|40471|            40471|
|   mean|          null|  3.34765634651973|47.428405524943784|55.891057794470115| null|             null|
| stddev|          null|1.6418850112078012| 117.5376337068714|127.40280622961957| null|             null|
|    min|R1005PFAPGMYQS|                 1|                10|                20|    N|                N|
|    max| RZZTUIX1574RC|                 5|             10498|             10780|    N|                Y|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+



Conclusion- There seems to be a bias among "Star Ratings", as there are 40,471 records that did not have vine reviews, on the other hand 94 records have vine reviews.

In [20]:
# Get percentage of 5-star rating among those records that had vine reviews
paid_five_star_number = vine_yes_review_df[vine_yes_review_df['star_rating']== 5].count()
paid_number = vine_yes_review_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

94
48
0.5106382978723404


In [21]:
# Get percentage of 5-star rating among those record that did not have vine reviews
paid_five_star_number = vine_no_review_df[vine_no_review_df['star_rating']== 5].count()
paid_number = vine_no_review_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)
     

40471
15663
0.38701786464381904


Conclusion- 51.06% of the Vine program gave a 5-star rating, whereas 38.70% of who had no review also gave a 5-star rating. Based on these numbers there is the possibility of biased in the Vine/Star-Rating reviews.